**In this part we take the matrix we created for BL count IR2 and make a matrix with new PIDs**



In [1]:
import pandas as pd
import numpy as np
from pathlib import Path
import functools

In [9]:
!pip install dask

Here we upload the the big diagnosis that we have created previously.

In [2]:
final2=pd.read_csv("final.csv", header=None,index_col=[0])

#we make it readable order that we need to have with making the columns in appropriate way.

final1=final2.rename(columns=final2.iloc[0]).drop(final2.index[0]).reset_index(drop=True)
final1

,PATNO,AGE_AT_VISIT,AFICBERB,ASHKJEW,BASQUE,BIRTHDT,SEX,CHLDBEAR,HANDED,HISPLAT,RAASIAN,RABLACK,RAHAWOPI,RAINDALS,RANOS,RAWHITE,COHORT_DEFINITION
0,3000,69.1,0,0,0,12/1941,0,0,1,0,0,0,0,0,0,1,Healthy Control
1,3001,65.1,0,0,0,01/1946,1,NaN,2,0,0,0,0,0,0,1,Parkinson's Disease
2,3002,67.6,0,0,0,08/1943,0,0,1,0,0,0,0,0,0,1,Parkinson's Disease
3,3003,56.7,0,0,0,07/1954,0,0,1,0,0,0,0,0,0,1,Parkinson's Disease
4,3004,59.4,0,0,0,11/1951,1,NaN,1,0,0,0,0,0,0,1,Healthy Control
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1207,160252,80.3,0,0,0,05/1942,1,NaN,1,0,0,0,0,0,0,1,Parkinson's Disease
1208,161236,55.6,0,0,0,01/1967,1,NaN,1,0,0,0,0,0,0,1,Parkinson's Disease
1209,162140,68.8,0,0,0,10/1953,1,NaN,1,0,0,0,0,0,0,1,Prodromal
1210,162905,45.2,0,0,0,06/1977,1,NaN,1,0,0,1,0,0,0,0,Healthy Control


In [7]:
path_data=Path("/scratch/znazari/matrix_g/new_jobs_counts_ir2/final_data_count/")

We read off the raw matrix that we have created previously, they usually contain more patient that we have their diagnosis.

In [8]:
bl_matrix=pd.read_csv(path_data/"bl_matrix_counts_ir2.txt", header=None, low_memory=False)


FileNotFoundError: [Errno 2] No such file or directory: '/scratch/znazari/matrix_g/new_jobs_counts_ir2/final_data_count/bl_matrix_counts_ir2.txt'

We resolve some problem that we have encoutered for reading the file

In [ ]:
bl_matrix2=bl_matrix.rename(columns=bl_matrix.iloc[0]).drop(bl_matrix.index[0]).reset_index(drop=True)

In [ ]:
bl_matrix2['Geneid']

Then we read the list of pateint that we have which is big and their diagnosis is not necessary available in the diagnosis matrix.

In [ ]:
path1=Path("/scratch/znazari/PPMI_ver_sep2022/RNA_Seq_data/star_ir2/counts/")

In [ ]:
bl_files=pd.read_csv(path1/"bl.txt", header=None )

In [ ]:
def function_names(fname):
    tokens=fname.split('.')
    return tokens[1]

Below we will have the list of the patient IDs that are still larger than what is supposed to be.

In [ ]:
bl_list=[]
for i in range(len(bl_files)):
    bl_list=bl_list+[function_names(bl_files.iloc[i][0])]

Below we ar going to have all the patients that we have their IDs and diagnosis.

In [ ]:
patient_diagonises=final1[['PATNO','COHORT_DEFINITION']]
patient_diagonises

In [ ]:
big_patients=list(patient_diagonises['PATNO'])
len(big_patients)

Below we are going to check how many of those patients that we initially get from the files of the RNA-seq are actually have their diagnosis in the big list of patients with their diagnosis.

In [ ]:
contains=[]
not_contains=[]
for i in bl_list:
    if i in big_patients:
        contains+=[i]
    else:
        not_contains+=[i]

Below we neet to get all the patient IDs which are represented as "string" to be presented as type integer, as it was for the list that we had in the orginal RNA-seq matrices.

In [ ]:
r_contains=[int(x) for x in contains]

Below we just extract the list of patients that are in the big original diagnosis matrix.

In [ ]:
bl_matrix3=bl_matrix2[r_contains]

Below we need to replace the geneID column that were missing previously.

In [ ]:
bl_matrix3.insert(loc=0, column ='Geneid',value = bl_matrix2['Geneid'])

In [ ]:
bl_matrix3.head(20)


In [ ]:
len(bl_matrix3)

In [ ]:
def f_names(fname):
    tokens=fname.split('.')
    return tokens[0]

In [ ]:
%%time
#enhanced =D
for i in range(len(bl_matrix3)):
    x_x=bl_matrix3.loc[i,"Geneid"]
    j_j=x_x.split('.')
    y_y=j_j[0]
    bl_matrix3.loc[i, 'Geneid']=y_y
bl_matrix3    


#  filter the data with only choosing the protein coding+IncRNA genes 
Ivan Arisi kindly sent the file

In [ ]:
path_i=Path("/home/znazari/ppmi_files/")

protein_ivan=pd.read_csv(path_i/"HUGO_protein_coding_genes_plus_high_confidence_lncRNA.txt",sep='\t', header=None)
protein_ivan.columns = ["Ensembl gene ID"]
protein_ivan

Below we try to find the indices of the genes that are actually in the protein coding genes file
we also get the container which contains those geneIDs < this is not necessary though.

In [ ]:
%%time
container=[]
cpy_bl=bl_matrix3.copy()
lis_=[]
for i in range(len(cpy_bl)):
    if cpy_bl.loc[i,"Geneid"] in list(protein_ivan.loc[:,'Ensembl gene ID']):
        lis_+=[i]
        container += [cpy_bl.loc[i,"Geneid"]]        
       
len(container)

In [ ]:
# we then try to figute out what indices of the genes IDs need to be dropped by subtracting the lists
large_lis=list(range(0,57820))
res=list(set(large_lis)-set(lis_))
res;

In [ ]:
# here we check if it did it correctly:
len(large_lis)-len(res)

Below we drop the gene IDs based on their indices that we figuered out previously and it is SUPER FAST.

In [ ]:
%%time
df_matrix3 = cpy_bl[~cpy_bl.index.isin(res)].copy()


In [ ]:
b_df_matrix3=df_matrix3.copy()
b_df_matrix3.reset_index(drop=True, inplace=True)
b_df_matrix3

Finally we write the result into a txt file.

In [ ]:
path_data=Path("/scratch/znazari/matrix_generation/new_jobs_counts_ir2/final_data_count/")

In [ ]:
b_df_matrix3.to_csv(path_data/"p_ordered_bl_matrix_ir2_count.txt")

In [ ]:
ml_path=Path("/home/znazari/machine_learning_ppmi")
b_df_matrix3.to_csv(ml_path/"p_ordered_bl_matrix_ir2_count.txt")
